In [1]:
import datetime
import logging
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

from citymobil_python_clickhouse_wrapper import ClickHouseWrapper
import pyexasol

In [2]:
cred = pd.read_json(r'/Users/skostuchik/crd_exa.json')
user = cred.login.squeeze()
password = cred.password_exa.squeeze()

In [3]:
# initialize ch
logger = logging.getLogger()
clickhouse = ClickHouseWrapper(
    logger,
    url="http://chmain-vip.local.city-dt.ru:8123",
    user=user,
    password=password,
    create_and_assign_event_loop=False,  # for django threads with no event loop
    allow_nested_event_loops=True  # for jupyter threads with existed event loop
)

In [4]:
# initialize exasol
C = pyexasol.connect(dsn='ex1..3.city-srv.ru:8563', user=user, password=password, fetch_dict=True)

In [5]:
#for i in tqdm(range(28)):
    query = f'''
              select h3,
                     DAYOFWEEK(date) wd,
                     case when extract(minute from dt) >= 30
                         then date_trunc('hour', dt) + interval 30 minute
                         else date_trunc('hour', dt) end dt_slot,
                     max(open_drivers_corrected) as drivers,
                     max(sum_uViews)             as uViews
              from surge_statistic.surge_20_data s
                       join exasol_mart.tariff t on toUInt64(s.tariff_id) = toUInt64(t.tariff_rk)
              where 1 = 1
              and date = toDate('2021-09-12') - interval '{i}' day
              and t.locality_rk = 22534
              and t.tariff_group_rk = 2
        group by h3, wd, dt_slot
    '''

    df = clickhouse.fetch(query)
    df = df[['h3', 'dt_slot', 'wd', 'drivers', 'uViews']]
    
    C.import_from_pandas(df, ('SANDBOX', 'd2v_voronezh_temp'))

100%|██████████| 28/28 [02:38<00:00,  5.66s/it]


***

In [6]:
v2_q = '''
select gch.CAMPAIGN_ID, gch.CAMPAIGN_NAME, week_day,
    concat(
       case when FROM_HOUR <= 9 then concat('0', FROM_HOUR) else FROM_HOUR end, ':',
       case when FROM_MINUTE <= 9 then concat('0', FROM_MINUTE) else FROM_MINUTE end) time_slot, gch.POLYGONS
from replica.geominimal_change_history gch
where gch.ID_LOCALITY = 22534
    and (gch.TIME_END is null or to_date(gch.TIME_END) > current_date)
    and gch.ACTIVE = 1
'''

In [7]:
voronezh_gch = pd.DataFrame(C.execute(v2_q).fetchall())

In [9]:
voronezh_gch = voronezh_gch.dropna()

In [10]:
#voronezh_gch.WEEK_DAY = voronezh_gch.WEEK_DAY.astype(int)

In [11]:
df = pd.DataFrame(columns = ['WEEK_DAY','TIME_SLOT','CAMPAIGN_ID','CAMPAIGN_NAME','polygon_id',
                             'polygon_name','price','coords'])

In [12]:
for index, row  in voronezh_gch.iterrows():
    for i in range(len(json.loads(row.POLYGONS))):
        df = df.append({'WEEK_DAY' : row.WEEK_DAY, 'TIME_SLOT' : row.TIME_SLOT,
                        'CAMPAIGN_ID' : row.CAMPAIGN_ID, 'CAMPAIGN_NAME' : row.CAMPAIGN_NAME,
                        'polygon_id' : json.loads(row.POLYGONS)[i].get('id'),
                        'polygon_name' : json.loads(row.POLYGONS)[i].get('name'),
                        'price' : json.loads(row.POLYGONS)[i].get('price'),
                        'coords' : str(json.loads(row.POLYGONS)[i].get('coords'))}, ignore_index=True)

In [13]:
df.coords = df.coords.apply(lambda x: x.replace("'",""))

In [14]:
df

,WEEK_DAY,TIME_SLOT,CAMPAIGN_ID,CAMPAIGN_NAME,polygon_id,polygon_name,price,coords
0,1,08:30,2611,Воронеж Утро,3014,Воронеж Северо-Запад,280,"[[51.712457652293, 39.162861881594], [51.70667..."
1,1,08:30,2611,Воронеж Утро,4302,Воронеж Север Ю,280,"[[51.698016998035, 39.182511280174], [51.69912..."
2,1,08:30,2611,Воронеж Утро,3002,Воронеж Левый берег ЖД район С,320,"[[51.698395700951, 39.273198149475], [51.69527..."
3,1,08:30,2611,Воронеж Утро,6403,Воронеж Левый берег ЖД район Ю,180,"[[51.686697695883, 39.25868202485], [51.687638..."
4,1,08:30,2611,Воронеж Утро,7319,Воронеж Юго-Запад В,180,"[[51.649779152197, 39.165467114521], [51.65097..."
...,...,...,...,...,...,...,...,...
365,5,21:00,2659,Воронеж поздний вечер,3006,Левый берег Вогрэс-Чернавский,250,"[[51.636942447984, 39.23075479483], [51.640467..."
366,5,21:00,2659,Воронеж поздний вечер,3010,центр малый полигон,250,"[[51.683955843786, 39.183265703861], [51.69542..."
367,5,21:00,2659,Воронеж поздний вечер,3018,центр большой полигон,250,"[[51.649829438075, 39.180566486818], [51.65195..."
368,5,21:00,2659,Воронеж поздний вечер,4302,Север,250,"[[51.698016998035, 39.182511280174], [51.69912..."


In [15]:
df['ID'] = df.index + 1

In [16]:
df = df[['ID','WEEK_DAY', 'TIME_SLOT', 'CAMPAIGN_ID', 'CAMPAIGN_NAME', 'polygon_id',
         'polygon_name', 'price', 'coords']]

In [17]:
C.import_from_pandas(df, ('SANDBOX', 'd2v_voronezh_gch')
                     #, import_params = {'columns':list(score.columns)}
                    )